In [1]:
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
directory = "D:\\#Data\\Scraped Data\\Shiksha"
filename = "bits_pilani.csv"
question_data = pd.read_csv(f"{directory}\\{filename}")

tag_data = pd.read_csv("D:\\#Data\\Rough Data\\tag_list.csv")

In [3]:
question_data.shape

(1850, 3)

In [4]:
tag_data.shape

(171, 2)

In [5]:
def match_keywords(question, keywords):
    matched_keywords = []
    matched_categories = set()

    if re.search(r"\bIT\b", question):
        matched_keywords.append("IT")
        matched_categories.add("courses")

    if re.search(r"\bCS\b", question):
        matched_keywords.append("CS")
        matched_categories.add("courses")

    if re.search(r"\b\d{1,3}%\b", question):
        matched_keywords.append("%")
        matched_categories.add("admissions")

    if re.search(r"percentile[?.,]?", question, re.IGNORECASE):
        matched_keywords.append("percentile")
        matched_categories.add("admissions")

    for index, row in keywords.iterrows():
        # Avoid adding "IT", "CS" again if it's already added
        if row["keyword"].lower() == "it" or row["keyword"].lower() == "cs":
            continue
        elif re.search(
            r"\b{}\b".format(re.escape(row["keyword"])), question, re.IGNORECASE
        ):
            matched_keywords.append(row["keyword"])
            matched_categories.add(row["subcategory"])

    return ", ".join(matched_keywords), ", ".join(matched_categories)

In [6]:
question_data['views'] = pd.to_numeric(question_data['views'], errors='coerce')
question_data['views'].fillna(0, inplace=True)
question_data['views'] = question_data['views'].astype(int)

In [10]:
question_data["keyword"], question_data["category"] = zip(
    *question_data["question"].apply(lambda x: match_keywords(x, tag_data))
)

In [11]:
question_data.loc[question_data['keyword'] == '', 'category'] = ''

In [12]:
question_data.head(15)

,question,views,posted,keyword,category
0,What is the placement record of LPU compared t...,1000,2 weeks ago,"stream, CSE, placement","placements, courses"
1,I want to get admission to BITs but my PCM agg...,34,22 hours ago,"admission, PCM",admissions
2,What is the BITS Pilani Goa BSc registration fee?,5,a week ago,"fee, BSc, registration","courses, admissions"
3,Does BITS Pilani Goa offers any internship for...,5,a week ago,"course, BSc, internship","placements, courses"
4,How can I prepare for BITS without taking help...,35,Yesterday,prepare for,admissions
5,Does BITS Pilani offers scholarships?,0,a week ago,scholarships,scholarships
6,What are the eligibility criteria for admissio...,10,2 weeks ago,"admission, eligibility",admissions
7,"Is BITS, Goa campus better than the BITS, Pila...",517,a month ago,,
8,Is JEE Main enough for BITS Pilani BTech admis...,539,a month ago,"BTech, admission, JEE","cutoff, courses, admissions"
9,Can I get admission to BSc without any entranc...,2,a week ago,"BSc, admission, exam, entrance exam","courses, admissions"


In [13]:
question_data.shape

(1850, 5)

In [14]:
empty_or_nan_keywords = (
    question_data["keyword"].apply(lambda x: x in [None, "", " "]).sum()
)
print(f"Number of empty or NaN entries in 'keyword' column: {empty_or_nan_keywords}")

Number of empty or NaN entries in 'keyword' column: 261


In [15]:
save_directory = "D:\\#Data\\Scraped Data\\Shiksha"
filename = "bits_pilani_questions_tagged.csv"
question_data.to_csv(f"{save_directory}\\{filename}", index=False)